In [1]:
import numpy as np
import keras
import os
from pathlib import Path

2025-09-25 08:31:24.037272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
batch_size = 64 
epochs = 100  
latent_dim = 256
num_samples = 10000 
data_path = "/home/mca/Downloads/hin-eng/hin.txt"

In [13]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 3116
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 107
Max sequence length for outputs: 123


In [14]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

2025-09-25 08:41:58.745458: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [15]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
39/39 [==============================] - 22s 512ms/step - loss: 1.2851 - accuracy: 0.7860 - val_loss: 1.8536 - val_accuracy: 0.6843
Epoch 2/100
39/39 [==============================] - 26s 658ms/step - loss: 1.0021 - accuracy: 0.8068 - val_loss: 1.9864 - val_accuracy: 0.6843
Epoch 3/100
39/39 [==============================] - 30s 778ms/step - loss: 0.9013 - accuracy: 0.8089 - val_loss: 1.4362 - val_accuracy: 0.6867
Epoch 4/100
39/39 [==============================] - 17s 446ms/step - loss: 0.8481 - accuracy: 0.8090 - val_loss: 1.4997 - val_accuracy: 0.6867
Epoch 5/100
39/39 [==============================] - 22s 562ms/step - loss: 0.8165 - accuracy: 0.8092 - val_loss: 1.4025 - val_accuracy: 0.6867
Epoch 6/100
39/39 [==============================] - 21s 523ms/step - loss: 0.8087 - accuracy: 0.8091 - val_loss: 1.3210 - val_accuracy: 0.6859
Epoch 7/100
39/39 [==============================] - 25s 643ms/step - loss: 0.7935 - accuracy: 0.8089 - val_loss: 1.3278 - val_accuracy:

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, 70)]           0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None, 93)]           0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 256),                334848    ['input_1[0][0]']             
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                              

In [17]:
model.save("LSTM_model.keras")

In [18]:
model = keras.models.load_model("LSTM_model.keras")

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq, verbose=0)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [19]:
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Wow!
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Duck!
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Duck!
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Help!
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Jump.
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Jump.
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Jump.
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Hello!
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Hello!
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Cheers!
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Cheers!
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Exhale.
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
Input sentence: Exhale.
Decoded sentence: मैं तुम्हारे साथ सात का किता है।

-
I

In [25]:
from tensorflow.keras.optimizers import RMSprop

def train_model(model, name):
    model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=64,
        epochs=30,
        validation_split=0.2
    )
    model.save(f"{name}_model.keras")
    return history

In [32]:
from tensorflow.keras.layers import GRU

def build_gru_model():
    encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
    encoder = GRU(256, return_state=True)
    encoder_outputs, state_h = encoder(encoder_inputs)
    encoder_states = [state_h]

    decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
    decoder_gru = GRU(256, return_sequences=True, return_state=True)
    decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(num_decoder_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model

In [33]:
gru_model = build_gru_model()
gru_history = train_model(gru_model, "gru")

Epoch 1/30
39/39 [==============================] - 11s 236ms/step - loss: 1.3421 - accuracy: 0.7858 - val_loss: 1.7311 - val_accuracy: 0.6843
Epoch 2/30
39/39 [==============================] - 9s 220ms/step - loss: 0.9463 - accuracy: 0.8065 - val_loss: 1.4296 - val_accuracy: 0.6843
Epoch 3/30
39/39 [==============================] - 10s 259ms/step - loss: 0.8814 - accuracy: 0.8069 - val_loss: 1.3984 - val_accuracy: 0.6864
Epoch 4/30
39/39 [==============================] - 14s 352ms/step - loss: 0.8235 - accuracy: 0.8082 - val_loss: 1.3221 - val_accuracy: 0.6814
Epoch 5/30
39/39 [==============================] - 14s 360ms/step - loss: 0.7826 - accuracy: 0.8089 - val_loss: 1.3103 - val_accuracy: 0.6868
Epoch 6/30
39/39 [==============================] - 13s 337ms/step - loss: 0.7405 - accuracy: 0.8169 - val_loss: 1.2054 - val_accuracy: 0.7028
Epoch 7/30
39/39 [==============================] - 13s 338ms/step - loss: 0.6968 - accuracy: 0.8267 - val_loss: 1.1484 - val_accuracy: 0.7115


In [34]:
gru_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, None, 70)]           0         []                            
                                                                                                  
 input_12 (InputLayer)       [(None, None, 93)]           0         []                            
                                                                                                  
 gru_6 (GRU)                 [(None, 256),                251904    ['input_11[0][0]']            
                              (None, 256)]                                                        
                                                                                                  
 gru_7 (GRU)                 [(None, None, 256),          269568    ['input_12[0][0]',      

In [48]:
model = keras.models.load_model("gru_model.keras")

latent_dim = 256
encoder_inputs = model.input[0]

encoder_gru = model.layers[2]
_, state_h_enc = encoder_gru.output

encoder_states = [state_h_enc]

encoder_model = keras.Model(encoder_inputs, encoder_states)
decoder_inputs = model.input[1]

decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_states_inputs = [decoder_state_input_h]

decoder_gru = model.layers[3]

decoder_outputs, state_h_dec = decoder_gru(
    decoder_inputs, initial_state=decoder_states_inputs
)

decoder_states = [state_h_dec]

decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, len(target_token_index)), dtype=np.float32)
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""

    while not stop_condition:
        output_tokens, h = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == "\n" or
            len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, len(target_token_index)), dtype=np.float32)
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h]

    return decoded_sentence

print("encoder_input_data shape:", encoder_input_data.shape)

for seq_index in range(20):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

encoder_input_data shape: (3116, 107, 70)
1/1 [==============================] - 0s 198ms/step


ValueError: operands could not be broadcast together with shapes (1,1,1,93) (1,256) 

In [46]:
print("encoder_input_data shape:", encoder_input_data.shape)  # should be (num_samples, max_encoder_seq_length, num_encoder_tokens)
print("input_seq shape for seq_index=0:", encoder_input_data[0:1].shape)  # should be (1, max_encoder_seq_length, num_encoder_tokens)

encoder_input_data shape: (3116, 107, 70)
input_seq shape for seq_index=0: (1, 107, 70)


In [47]:
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 193ms/step
target_seq ndim: 3
target_seq shape: (1, 1, 93)
states_value ndim: [1]
states_value shapes: [(256,)]


ValueError: operands could not be broadcast together with shapes (1,1,1,93) (1,256) 